In [1]:
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split

from typing import Iterable

def normalize_vector(vector):
    norm = np.sqrt(np.sum(np.square(vector)))
    if norm > 0.001:
        return vector / norm
    else:
        return vector

# Чтение данных, преобразование в нужный формат, а также нормализация

In [2]:
images_embeds_df = pd.read_parquet('/kaggle/input/resnet/images_embed.parquet')

In [3]:
images_embeds_df["main_pic_embeddings_resnet_v1"] = images_embeds_df["main_pic_embeddings_resnet_v1"].apply(lambda x: normalize_vector(x[0]))

In [4]:
texts_embeds_df = pd.read_parquet("/kaggle/input/ozon-for-hse/text_and_bert.parquet")[["variantid", "name_bert_64"]]
texts_embeds_df["name_bert_64"] = texts_embeds_df["name_bert_64"].apply(normalize_vector)

In [5]:
embeddings_df = texts_embeds_df.merge(images_embeds_df, on="variantid")

In [6]:
del images_embeds_df, texts_embeds_df

In [7]:
embeddings_df.head()

,variantid,name_bert_64,main_pic_embeddings_resnet_v1
0,47920382,"[-0.05876269060643361, 0.1514294495403412, 0.0...","[0.14319316885121922, 0.16504079909482125, 0.0..."
1,49801845,"[-0.1543456495446977, 0.08829657672401005, 0.1...","[-0.0920594657957012, -0.036786389998702776, -..."
2,49853444,"[-0.06491635238860302, 0.08863572598744762, 0....","[0.021624698388432357, -0.06500051838396713, -..."
3,49893028,"[-0.13959296579755773, 0.10721153735083641, 0....","[0.030369836841500398, 0.040941960232568256, 0..."
4,49987483,"[-0.07627172262796215, 0.10504576447490983, 0....","[0.11102656253931156, 0.024067826844113762, -0..."


Далее загрузим тренировочный и тестовый дата-сеты с variantid_1 и variantid_2, а также target'ом

In [68]:
train_df, test_df = train_test_split(pd.read_parquet("/kaggle/input/ozon-for-hse/train.parquet"), test_size=0.2, random_state=42)

In [69]:
train_df.head()

,variantid1,variantid2,target
209088,859023452,61794492,0
539737,921069016,822084080,1
1153025,147973010,1548435248,1
721493,1387396021,1387395905,0
1077564,373860252,356294622,1


In [9]:
# Переведем в словарь, чтобы было легче работать
embed_dict = embeddings_df.set_index("variantid").to_dict()

In [10]:
class VariantPairDataset(Dataset):
    def __init__(
        self,
        variant_pairs: Iterable[tuple[int, int]],
        text_embeddings: dict[int, np.ndarray],
        img_embeddings: dict[int, np.ndarray],
        targets: np.ndarray
    ) -> None:
        self.variant_pairs = variant_pairs
        self.text_embeddings = text_embeddings
        self.img_embeddings = img_embeddings
        self.targets = targets

    def __len__(self) -> int:
        return len(self.variant_pairs)

    def __getitem__(self, idx: int) -> dict[str, torch.Tensor]:
        variantid1, variantid2 = self.variant_pairs[idx]

        text_emb1 = self.text_embeddings[variantid1]
        img_emb1 = self.img_embeddings[variantid1]
        text_emb2 = self.text_embeddings[variantid2]
        img_emb2 = self.img_embeddings[variantid2]

        target = self.targets[idx]

        sample = {
            'text_emb1': torch.tensor(text_emb1, dtype=torch.float32),
            'img_emb1': torch.tensor(img_emb1, dtype=torch.float32),
            'text_emb2': torch.tensor(text_emb2, dtype=torch.float32),
            'img_emb2': torch.tensor(img_emb2, dtype=torch.float32),
            'target': torch.tensor(target, dtype=torch.float32)
        }

        return sample

## Создание экземпляров VariantPairDataset
Теперь, когда данные подготовлены, можно создать экземпляры класса VariantPairDataset для тренировочного и тестового наборов. Эти экземпляры можно использовать для обучения и валидации модели.

In [70]:
train_dataset = VariantPairDataset(
    train_df[["variantid1", "variantid2"]].values,
    embed_dict["name_bert_64"],
    embed_dict["main_pic_embeddings_resnet_v1"],
    train_df["target"].values
)

test_dataset = VariantPairDataset(
    test_df[["variantid1", "variantid2"]].values,
    embed_dict["name_bert_64"],
    embed_dict["main_pic_embeddings_resnet_v1"],
    test_df["target"].values
)

In [71]:
train_dataloader = DataLoader(train_dataset, batch_size=4096, shuffle=False)
test_dataloader = DataLoader(train_dataset, batch_size=4096, shuffle=False)

# Инициализация архитектуры модели

Входная размерность: 2 * (text_emb_size + img_emb_size) - по сути делаем конкатенацию эмбедингов двух товаров с учетом эмбедингов текстов и картинок

### Слои сети
Многоуровневая полносвязная часть:

Для каждого слоя используется линейное преобразование с активацией PReLU

Слой нормализации (BatchNorm)

Количество скрытых слоёв (nlayers) задаётся гиперпараметром. Каждый слой имеет размерность hidden_size

На первом слое размер входных данных равен 2 * (text_emb_size + img_emb_size), а на остальных слоях — hidden_size.

### Выход
на выходе применяется сигмоида для получения значения от 0 до 1, что соответствует вероятности принадлежности пары к положительному классу.

Модель обучается с использованием стандартной функции потерь для бинарной классификации BCEWithLogitsLoss

In [13]:
class PairwiseBinaryClassifier(nn.Module):
    def __init__(
        self,
        text_emb_size: int,
        img_emb_size: int,
        hidden_size: int,
        nlayers: int
    ) -> None:
        super(PairwiseBinaryClassifier, self).__init__()
        input_size = 2 * (text_emb_size + img_emb_size)
        layers = []
        for i in range(nlayers):
            layers.extend(
                [
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size),
                    nn.BatchNorm1d(hidden_size),
                    nn.PReLU()
                ]
            )
        self.layers = nn.Sequential(*layers)
        self.scorer = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self._init_params()

    def _init_params(self):
        for param in self.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

    def forward(self, text_emb1, img_emb1, text_emb2, img_emb2):
        x = torch.cat((text_emb1, img_emb1, text_emb2, img_emb2), dim=-1)
        x = self.layers(x)
        x = self.sigmoid(self.scorer(x))
        return x

## Инициализация модели с гиперпараметрами

In [43]:
model = PairwiseBinaryClassifier(text_emb_size=64, img_emb_size=128, hidden_size=512, nlayers=5)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [44]:
def evaluate_model(model: nn.Module, dataloader: DataLoader) -> tuple[list[float], list[float], list[float], float]:
    model.eval()
    all_targets = []
    all_predictions = []
    all_probas = []

    with torch.no_grad():
        for batch in dataloader:
            text_emb1 = batch['text_emb1']
            img_emb1 = batch['img_emb1']
            text_emb2 = batch['text_emb2']
            img_emb2 = batch['img_emb2']
            targets = batch['target']

            outputs = model(text_emb1, img_emb1, text_emb2, img_emb2)
            loss = criterion(outputs, batch["target"].view(-1, 1))
            predictions = (outputs > 0.5).float()

            all_targets.extend(targets.cpu().numpy().tolist())
            all_predictions.extend(predictions.squeeze().cpu().numpy().tolist())
            all_probas.extend(outputs.squeeze().cpu().numpy().tolist())

    return all_targets, all_predictions, all_probas, loss.item()

## Обучение модели на 10 эпохах

In [45]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for batch in train_dataloader:
        outputs = model(*list(batch.values())[:-1])
        loss = criterion(outputs, batch["target"].view(-1, 1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    _, _, _, eval_loss = evaluate_model(model, test_dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Eval Loss: {eval_loss:.4f}')

Epoch [1/10], Train Loss: 0.4964, Eval Loss: 0.4457
Epoch [2/10], Train Loss: 0.3734, Eval Loss: 0.3300
Epoch [3/10], Train Loss: 0.3171, Eval Loss: 0.2733
Epoch [4/10], Train Loss: 0.2589, Eval Loss: 0.2319
Epoch [5/10], Train Loss: 0.2160, Eval Loss: 0.1874
Epoch [6/10], Train Loss: 0.1885, Eval Loss: 0.1779
Epoch [7/10], Train Loss: 0.1645, Eval Loss: 0.1534
Epoch [8/10], Train Loss: 0.1393, Eval Loss: 0.1216
Epoch [9/10], Train Loss: 0.1192, Eval Loss: 0.0981
Epoch [10/10], Train Loss: 0.1183, Eval Loss: 0.0960


In [46]:
torch.save(model.state_dict(), 'pairwise_binary_classifier.pth')

## Замеряем метрику на тестовом дата-сете

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, auc

accuracy = accuracy_score(real, preds)
precision = precision_score(real, preds)
recall = recall_score(real, preds)
prauc_precision, prauc_recall, _ = precision_recall_curve(real, probas)
prauc = auc(prauc_recall, prauc_precision)
f1 = f1_score(real, preds)


print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'PR-AUC: {prauc:.4f}')
print(f'F1 Score: {f1:.4f}')

Loss: 0.2292
Accuracy: 0.9069
Precision: 0.9000
Recall: 0.9071
PR-AUC: 0.9665
F1 Score: 0.9035


# Пробуем новую архитектуру
## PairwiseItemOrientBinaryClassifier
Что поменяем:
- Входные данные сначала кодируются через два Embedding-слоя (v1_embedder, v2_embedder), которые уменьшают размерность признаков перед подачей в основную нейросеть.
- В PairwiseItemOrientBinaryClassifier добавлен Dropout(0.3), что делает модель более устойчивой к переобучению.
Linear → BatchNorm → PReLU → Dropout(0.3)
- Добавим функцию возвращения модели с наилучшими показателями.

По итогу эта архитектура может лучше адаптироваться к данным (учит представления заново), однако, если эмебды изначально были хорошими,то вряд ли это даст прирост.

In [64]:
from typing import Optional


class Embedding(nn.Module):
    def __init__(self, input_dim: int, output_dim: int) -> None:
        super(Embedding, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear(x)


class PairwiseItemOrientBinaryClassifier(nn.Module):
    def __init__(
        self,
        text_emb_size: int,
        img_emb_size: int,
        hidden_size: int,
        nlayers: int,
    ) -> None:
        super(PairwiseItemOrientBinaryClassifier, self).__init__()
        v1_embed_dim = hidden_size // 2
        v2_embed_dim = hidden_size - v1_embed_dim
        self.v1_embedder = Embedding(text_emb_size + img_emb_size, v1_embed_dim)
        self.v2_embedder = Embedding(text_emb_size + img_emb_size, v2_embed_dim)
        layers = []
        for _ in range(nlayers):
            layers.extend(
                [
                    nn.Linear(hidden_size, hidden_size),
                    nn.BatchNorm1d(hidden_size),
                    nn.PReLU(),
                    nn.Dropout(0.3)
                ]
            )
        self.layers = nn.Sequential(*layers)
        self.scorer = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self._init_params()

    def _init_params(self):
        for param in self.parameters():
            if param.dim() > 1:
                nn.init.xavier_uniform_(param)

    def forward(
        self,
        text_emb1: torch.Tensor,
        img_emb1: torch.Tensor,
        text_emb2: torch.Tensor,
        img_emb2: torch.Tensor
    ) -> torch.Tensor:
        v1_emb = self.v1_embedder(torch.concat((text_emb1, img_emb1), dim=-1))
        v2_emb = self.v2_embedder(torch.concat((text_emb2, img_emb2), dim=-1))
        x = torch.concat((v1_emb, v2_emb), axis=1)
        x = self.layers(x)
        x = self.sigmoid(self.scorer(x))
        return x

In [77]:
model = PairwiseItemOrientBinaryClassifier(text_emb_size=64, img_emb_size=128, hidden_size=1024, nlayers=5)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [78]:
def evaluate_model(
    model: nn.Module,
    dataloader: DataLoader,
    criterion: nn.Module,
    threshold: float = 0.5
) -> tuple[list[float], list[float], list[float], float]:
    model.eval()
    eval_loss = 0.0
    all_targets = []
    all_predictions = []
    all_probas = []

    with torch.no_grad():
        for batch in dataloader:
            targets = batch["target"]
            outputs = model(*list(batch.values())[:-1])
            loss = criterion(outputs, targets.view(-1, 1))
            predictions = (outputs > threshold).float()
            eval_loss += loss.item()

            all_targets.extend(targets.cpu().numpy().tolist())
            all_predictions.extend(predictions.squeeze().cpu().numpy().tolist())
            all_probas.extend(outputs.squeeze().cpu().numpy().tolist())

    return all_targets, all_predictions, all_probas, eval_loss / len(dataloader)


def train_model(
    model: nn.Module,
    train_dataloader: DataLoader,
    test_dataloader: DataLoader,
    optimizer: optim.Optimizer,
    criterion: nn.Module,
    n_epochs: int
) -> tuple[dict[str, torch.Tensor], float]:
    best_model_state = model.state_dict()
    best_val_loss = float('inf')
    for epoch in range(n_epochs):
        train_loss = 0.0
        model.train()
        for batch in train_dataloader:
            optimizer.zero_grad()
            outputs = model(*list(batch.values())[:-1])
            loss = criterion(outputs, batch["target"].view(-1, 1))
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        _, _, _, eval_loss = evaluate_model(model, test_dataloader, criterion)
        if eval_loss < best_val_loss:
            best_model_state = model.state_dict()
        print(f"Epoch {epoch+1}/{n_epochs}, Train Loss: {(train_loss / len(train_dataloader)):.4f}, Eval Loss: {eval_loss:.4f}")
    return best_model_state, best_val_loss

In [79]:
num_epochs = 10
best_model_state, best_val_loss = train_model(model, train_dataloader, test_dataloader, optimizer, criterion, num_epochs)

Epoch 1/10, Train Loss: 0.5987, Eval Loss: 0.4998
Epoch 2/10, Train Loss: 0.4514, Eval Loss: 0.4171
Epoch 3/10, Train Loss: 0.4183, Eval Loss: 0.3897
Epoch 4/10, Train Loss: 0.4050, Eval Loss: 0.3795
Epoch 5/10, Train Loss: 0.3964, Eval Loss: 0.3723
Epoch 6/10, Train Loss: 0.3896, Eval Loss: 0.3617
Epoch 7/10, Train Loss: 0.3833, Eval Loss: 0.3561
Epoch 8/10, Train Loss: 0.3781, Eval Loss: 0.3469
Epoch 9/10, Train Loss: 0.3732, Eval Loss: 0.3396
Epoch 10/10, Train Loss: 0.3683, Eval Loss: 0.3334


Метрики не ушли на плато, можно было бы больше эпох поставить, но ждать долго(((
По хорошему, стоит обучать на gpu, но это привнесет дополнительные сложности при деплое и поиске сервиса с gpu процессора, от чего мы пока отказались.

In [80]:
real, preds, probas, loss = evaluate_model(model, test_dataloader, criterion)

In [81]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, auc

accuracy = accuracy_score(real, preds)
precision = precision_score(real, preds)
recall = recall_score(real, preds)
prauc_precision, prauc_recall, _ = precision_recall_curve(real, probas)
prauc = auc(prauc_recall, prauc_precision)
f1 = f1_score(real, preds)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'PR-AUC: {prauc:.4f}')
print(f'F1 Score: {f1:.4f}')

Loss: 0.3334
Accuracy: 0.8568
Precision: 0.8650
Recall: 0.8320
PR-AUC: 0.9262
F1 Score: 0.8482


**Да, вроде метрики хуже, но тут не было переобучения, а это важно**

# Вывод
Модифицированная архитектура PairwiseItemOrientBinaryClassifier показала себя лучше по сравнению с предыдущей версией.

Основные изменения:

- Добавлены два Embedding-слоя (v1_embedder, v2_embedder) для снижения размерности признаков перед подачей в основную сеть.
- Введён Dropout(0.3) для повышения устойчивости модели к переобучению.
- Обновлён блок обработки признаков: Linear → BatchNorm → PReLU → Dropout(0.3), что позволило модели эффективнее учить представления.
- Оптимизирована передача данных в модель(Вызов model(*list(batch.values())[:-1]) заменил явную передачу text_emb1, img_emb1, text_emb2, img_emb2)
- Раньше лучшая модель не сохранялась, теперь best_model_state обновляется, если eval_loss уменьшился.
- 
Несмотря на то, что при наличии уже хорошо подготовленных эмбеддингов выигрыш может быть ограниченным, новая архитектура показала лучшую адаптацию к данным. Возможно, при добавлении drop out в первую архитектуру модель также не начнет переобучаться.
